### Question:
Generate the output as shown below, where for First_name, Second_name and Last_name columns will be generated by utilizing UDFs.

| DATE       | TITLE         | first_name | last_name | middle_name |
| ---------- | ------------- | ---------- | --------- | ----------- |
| 02-08-2017 | Councilmember | Jennifer   | Gates     | S.          |
| 02-08-2017 | Councilmember | Philip     | Kingston  | T.          |
| 02-08-2017 | Mayor         | Michael    | Rawlings  | S.          |
| 02-08-2017 | Councilmember | Adam       | Medrano   |             |
| 02-08-2017 | Councilmember | Casey      | Thomas    |             |
| 02-08-2017 | Councilmember | Carolyn    | Arnold    | King        |
| 02-08-2017 | Councilmember | Scott      | Griggs    |             |
| 02-08-2017 | Councilmember | B.         | McGough   | Adam        |
| 02-08-2017 | Councilmember | Lee        | Kleinman  |             |
| 02-08-2017 | Councilmember | Sandy      | Greyson   |             |
| 02-08-2017 | Councilmember | Jennifer   | Gates     | S.          |
| 02-08-2017 | Councilmember | Philip     | Kingston  | T.          |
| 02-08-2017 | Mayor         | Michael    | Rawlings  | S.          |
| 02-08-2017 | Councilmember | Adam       | Medrano   |             |
| 02-08-2017 | Councilmember | Casey      | Thomas    |             |
| 02-08-2017 | Councilmember | Carolyn    | Arnold    | King        |
| 02-08-2017 | Councilmember | Rickey     | Callahan  | D.          |
| 01-11-2017 | Councilmember | Jennifer   | Gates     | S.          |
| 04/25/2018 | Councilmember | Sandy      | Greyson   |             |
| 04/25/2018 | Councilmember | Jennifer   | Gates     | S.          |

In [1]:
pip install pyspark

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("how to read csv file").getOrCreate()

sc = spark.sparkContext

In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Define User defined functions to extract first name, middle name, and last name
def first_name(name):
    return name.split()[0] if name else None

def last_name(name):
    return name.split()[-1] if name else None

def middle_name(name):
    return ' '.join(name.split()[1:2]) if len(name.split()) > 2 else ' '

# load the .csv file
voter_df = spark.read.csv('DallasCouncilVoters.csv',header=True)

# Filtering out rows where VOTER_NAME is null
voter_df = voter_df.filter(voter_df['VOTER_NAME'].isNotNull())

# Creating UDFs
udfFirstName = F.udf(first_name, StringType())
udfMiddleName = F.udf(middle_name, StringType())
udfLastName = F.udf(last_name, StringType())

# Apply the UDFs to create columns
voter_df = voter_df.withColumn('first_name', udfFirstName(voter_df.VOTER_NAME))
voter_df = voter_df.withColumn('last_name', udfLastName(voter_df.VOTER_NAME))
voter_df = voter_df.withColumn('middle_name', udfMiddleName(voter_df.VOTER_NAME))

# Drop the columns
voter_df = voter_df.drop('splits', 'random_val', 'VOTER_NAME')

# Show the DataFrame
voter_df.show()

+----------+-------------+----------+---------+-----------+
|      DATE|        TITLE|first_name|last_name|middle_name|
+----------+-------------+----------+---------+-----------+
|02/08/2017|Councilmember|  Jennifer|    Gates|         S.|
|02/08/2017|Councilmember|    Philip| Kingston|         T.|
|02/08/2017|        Mayor|   Michael| Rawlings|         S.|
|02/08/2017|Councilmember|      Adam|  Medrano|           |
|02/08/2017|Councilmember|     Casey|   Thomas|           |
|02/08/2017|Councilmember|   Carolyn|   Arnold|       King|
|02/08/2017|Councilmember|     Scott|   Griggs|           |
|02/08/2017|Councilmember|        B.|  McGough|       Adam|
|02/08/2017|Councilmember|       Lee| Kleinman|           |
|02/08/2017|Councilmember|     Sandy|  Greyson|           |
|02/08/2017|Councilmember|  Jennifer|    Gates|         S.|
|02/08/2017|Councilmember|    Philip| Kingston|         T.|
|02/08/2017|        Mayor|   Michael| Rawlings|         S.|
|02/08/2017|Councilmember|      Adam|  M